# Exercício 4: Análise de Entropia e Ganho de Informação

**Objetivo**: Calcular entropia e ganho de informação para construção de árvores de decisão

**Requisitos do guião**:
1. Calcular entropia dos 4 conjuntos (completo + 3 subconjuntos)
2. Calcular ganho da partição
3. Calcular ganho para todas as features
4. Explicar construção de árvore de decisão

## Definição das Features do Dataset Iris

O dataset Iris contém 4 features medidas em centímetros:
- **Feature 1 (Sepal Length)**: Comprimento do sépala - parte externa da flor que protege o botão
- **Feature 2 (Sepal Width)**: Largura do sépala - largura da parte externa da flor
- **Feature 3 (Petal Length)**: Comprimento da pétala - parte colorida da flor
- **Feature 4 (Petal Width)**: Largura da pétala - largura da parte colorida da flor


In [6]:
import numpy as np
import math

def load_iris_data(filepath):
    """Carrega o dataset Iris"""
    data = []
    labels = []
    
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if line:
                parts = line.split(',')
                if len(parts) == 5:
                    features = [float(x) for x in parts[:4]]
                    label = parts[4]
                    data.append(features)
                    labels.append(label)
    
    X = np.array(data)
    unique_labels = sorted(list(set(labels)))
    label_to_num = {label: i for i, label in enumerate(unique_labels)}
    y = np.array([label_to_num[label] for label in labels])
    
    return X, y, unique_labels

def discretize_features(X, method='tercis'):
    """Discretiza features em low/medium/high"""
    X_discretized = np.zeros_like(X, dtype=int)
    thresholds = {}
    
    for feature_idx in range(X.shape[1]):
        feature_values = X[:, feature_idx]
        threshold_low = np.percentile(feature_values, 33.33)
        threshold_high = np.percentile(feature_values, 66.67)
        
        discretized_feature = np.zeros(len(feature_values), dtype=int)
        discretized_feature[feature_values <= threshold_low] = 0  # low
        discretized_feature[(feature_values > threshold_low) & (feature_values <= threshold_high)] = 1  # medium
        discretized_feature[feature_values > threshold_high] = 2  # high
        
        X_discretized[:, feature_idx] = discretized_feature
        thresholds[feature_idx] = (threshold_low, threshold_high)
    
    return X_discretized, thresholds

def calculate_entropy(y_binary):
    """Calcula entropia: entropy(S) = -p+ × log2(p+) - p- × log2(p-)"""
    if len(y_binary) == 0:
        return 0
    
    n_positive = np.sum(y_binary == 1)
    n_negative = np.sum(y_binary == 0)
    total = len(y_binary)
    
    p_positive = n_positive / total
    p_negative = n_negative / total
    
    entropy = 0
    if p_positive > 0:
        entropy -= p_positive * math.log2(p_positive)
    if p_negative > 0:
        entropy -= p_negative * math.log2(p_negative)
    
    return entropy

def calculate_information_gain(X_discretized, y_binary, feature_idx):
    """Calcula ganho: gain(S,a) = entropy(S) - Σ(|Sv| × entropy(Sv)) / |S|"""
    original_entropy = calculate_entropy(y_binary)
    feature_values = X_discretized[:, feature_idx]
    unique_values = np.unique(feature_values)
    
    weighted_entropy = 0
    total_samples = len(y_binary)
    subsets_info = {}
    
    for value in unique_values:
        subset_mask = (feature_values == value)
        subset_y = y_binary[subset_mask]
        subset_size = len(subset_y)
        subset_entropy = calculate_entropy(subset_y)
        
        # Fórmula: (|Sv| × entropy(Sv)) / |S|
        weighted_entropy += (subset_size * subset_entropy) / total_samples
        
        n_positive = np.sum(subset_y == 1)
        n_negative = np.sum(subset_y == 0)
        
        subsets_info[value] = {
            'size': subset_size,
            'entropy': subset_entropy,
            'n_positive': n_positive,
            'n_negative': n_negative
        }
    
    information_gain = original_entropy - weighted_entropy
    return information_gain, subsets_info

# Carregar e preparar dados
X_continuous, y, class_names = load_iris_data('iris/iris.data')
feature_names = ['Feature 1 (Sepal Length)', 'Feature 2 (Sepal Width)', 'Feature 3 (Petal Length)', 'Feature 4 (Petal Width)']

# Discretizar features
X_discretized, thresholds = discretize_features(X_continuous, method='tercis')

# Criar target binário: Iris-setosa (p+) vs. outras (p-)
y_binary = (y == 0).astype(int)  # 0 = Iris-setosa

print(f"Dataset: {X_continuous.shape[0]} exemplos, {X_continuous.shape[1]} features")
print(f"Classes: {class_names}")
print(f"Target binário: Iris-setosa (p+) vs. outras (p-)")
print(f"Distribuição: {np.sum(y_binary)} Iris-setosa, {len(y_binary) - np.sum(y_binary)} outras")


Dataset: 150 exemplos, 4 features
Classes: ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
Target binário: Iris-setosa (p+) vs. outras (p-)
Distribuição: 50 Iris-setosa, 100 outras


## 1. Cálculo da Entropia dos 4 Conjuntos

**Fórmula**: `entropy(S) = -p+ × log₂(p+) - p- × log₂(p-)`

Vamos calcular a entropia do conjunto completo e dos 3 subconjuntos criados pela primeira feature (Feature 1 - Sepal Length).


In [7]:
# 1. Entropia do conjunto completo
original_entropy = calculate_entropy(y_binary)
print(f"=== ENTROPIA DO CONJUNTO COMPLETO ===")
print(f"entropy(S) = {original_entropy:.4f}")

# 2. Entropia dos 3 subconjuntos (primeira feature - Feature 1)
feature_idx = 0  # Feature 1 (Sepal Length)
feature_values = X_discretized[:, feature_idx]
value_names = ['Low', 'Medium', 'High']

print(f"\n=== ENTROPIA DOS 3 SUBCONJUNTOS ({feature_names[feature_idx]}) ===")
print(f"Limiares: Low ≤ {thresholds[feature_idx][0]:.2f}, Medium ≤ {thresholds[feature_idx][1]:.2f}, High > {thresholds[feature_idx][1]:.2f}")

subsets_entropy = {}
for value in [0, 1, 2]:  # low, medium, high
    subset_mask = (feature_values == value)
    subset_y = y_binary[subset_mask]
    subset_entropy = calculate_entropy(subset_y)
    subsets_entropy[value] = subset_entropy
    
    n_positive = np.sum(subset_y == 1)
    n_negative = np.sum(subset_y == 0)
    
    print(f"\n{value_names[value]} Dataset:")
    print(f"  Tamanho: {len(subset_y)} exemplos")
    print(f"  Iris-setosa: {n_positive}, Outras: {n_negative}")
    print(f"  entropy(S_{value_names[value].lower()}) = {subset_entropy:.4f}")

print(f"\n=== RESUMO DAS 4 ENTROPIAS ===")
print(f"1. Conjunto completo (S): {original_entropy:.4f}")
print(f"2. Low Dataset: {subsets_entropy[0]:.4f}")
print(f"3. Medium Dataset: {subsets_entropy[1]:.4f}")
print(f"4. High Dataset: {subsets_entropy[2]:.4f}")


=== ENTROPIA DO CONJUNTO COMPLETO ===
entropy(S) = 0.9183

=== ENTROPIA DOS 3 SUBCONJUNTOS (Feature 1 (Sepal Length)) ===
Limiares: Low ≤ 5.40, Medium ≤ 6.30, High > 6.30

Low Dataset:
  Tamanho: 52 exemplos
  Iris-setosa: 45, Outras: 7
  entropy(S_low) = 0.5700

Medium Dataset:
  Tamanho: 56 exemplos
  Iris-setosa: 5, Outras: 51
  entropy(S_medium) = 0.4341

High Dataset:
  Tamanho: 42 exemplos
  Iris-setosa: 0, Outras: 42
  entropy(S_high) = 0.0000

=== RESUMO DAS 4 ENTROPIAS ===
1. Conjunto completo (S): 0.9183
2. Low Dataset: 0.5700
3. Medium Dataset: 0.4341
4. High Dataset: 0.0000


## 2. Cálculo do Ganho da Partição

**Fórmula**: `gain(S,a) = entropy(S) - Σ(|Sv| × entropy(Sv)) / |S|`

O ganho mede quanto a partição reduz a entropia média.


In [ ]:
# Calcular ganho para a primeira feature
gain, subsets_info = calculate_information_gain(X_discretized, y_binary, feature_idx)

print(f"=== GANHO DA PARTIÇÃO ({feature_names[feature_idx]}) ===")
print(f"\nCálculo detalhado:")
print(f"entropy(S) = {original_entropy:.4f}")

# Calcular entropia ponderada
weighted_entropy = 0
total_samples = len(y_binary)
print(f"\nEntropia ponderada dos subconjuntos:")
for value in [0, 1, 2]:
    if value in subsets_info:
        info = subsets_info[value]
        weight = info['size'] / total_samples
        weighted_contribution = (info['size'] * info['entropy']) / total_samples
        weighted_entropy += weighted_contribution
        
        print(f"  |S_{value_names[value].lower()}| × entropy(S_{value_names[value].lower()}) / |S| = {info['size']} × {info['entropy']:.4f} / {total_samples} = {weighted_contribution:.4f}")

print(f"\nΣ(|Sv| × entropy(Sv)) / |S| = {weighted_entropy:.4f}")
print(f"\ngain(S, {feature_names[feature_idx]}) = {original_entropy:.4f} - {weighted_entropy:.4f} = {gain:.4f}")



=== GANHO DA PARTIÇÃO (Feature 1 (Sepal Length)) ===

Cálculo detalhado:
entropy(S) = 0.9183

Entropia ponderada dos subconjuntos:
  |S_low| × entropy(S_low) / |S| = 52 × 0.5700 / 150 = 0.1976
  |S_medium| × entropy(S_medium) / |S| = 56 × 0.4341 / 150 = 0.1621
  |S_high| × entropy(S_high) / |S| = 42 × 0.0000 / 150 = 0.0000

Σ(|Sv| × entropy(Sv)) / |S| = 0.3596

gain(S, Feature 1 (Sepal Length)) = 0.9183 - 0.3596 = 0.5587

=== INTERPRETAÇÃO ===
• Antes da partição: entropia = 0.9183 (impureza moderada)
• Após partição: entropia média = 0.3596
• Redução: 0.5587 (60.8% de melhoria)
• Significado: Feature 1 (Sepal Length) reduz significativamente a incerteza na classificação


## 3. Ganho para Todas as Features

Vamos calcular o ganho para todas as 4 features e identificar a melhor.


In [9]:
print(f"=== GANHO PARA TODAS AS FEATURES ===")

gains = []
for feature_idx in range(len(feature_names)):
    gain, _ = calculate_information_gain(X_discretized, y_binary, feature_idx)
    gains.append((feature_idx, gain, feature_names[feature_idx]))

# Ordenar por ganho (maior primeiro)
gains.sort(key=lambda x: x[1], reverse=True)

print(f"\n{'Feature':<25} {'Ganho':<8} {'Ranking'}")
print("-" * 45)
for rank, (feature_idx, gain, feature_name) in enumerate(gains, 1):
    print(f"{feature_name:<25} {gain:<8.4f} {rank}º")

best_feature_idx, best_gain, best_feature_name = gains[0]
print(f"\n=== MELHOR FEATURE ===")
print(f"Feature: {best_feature_name}")
print(f"Ganho: {best_gain:.4f}")
print(f"\nSignificado: {best_feature_name} oferece o maior potencial de classificação")
print(f"para distinguir Iris-setosa das outras classes.")

# Mostrar análise da melhor feature
_, best_subsets = calculate_information_gain(X_discretized, y_binary, best_feature_idx)
print(f"\n=== ANÁLISE DA MELHOR FEATURE ===")
print(f"Particionamento por {best_feature_name}:")
for value in [0, 1, 2]:
    if value in best_subsets:
        info = best_subsets[value]
        print(f"  {value_names[value]}: {info['size']} exemplos, entropia = {info['entropy']:.4f}")
        if info['entropy'] == 0:
            if info['n_positive'] == info['size']:
                print(f"    → Todos são Iris-setosa (puro!)")
            else:
                print(f"    → Nenhum é Iris-setosa (puro!)")


=== GANHO PARA TODAS AS FEATURES ===

Feature                   Ganho    Ranking
---------------------------------------------
Feature 3 (Petal Length)  0.9183   1º
Feature 4 (Petal Width)   0.9183   2º
Feature 1 (Sepal Length)  0.5587   3º
Feature 2 (Sepal Width)   0.3081   4º

=== MELHOR FEATURE ===
Feature: Feature 3 (Petal Length)
Ganho: 0.9183

Significado: Feature 3 (Petal Length) oferece o maior potencial de classificação
para distinguir Iris-setosa das outras classes.

=== ANÁLISE DA MELHOR FEATURE ===
Particionamento por Feature 3 (Petal Length):
  Low: 50 exemplos, entropia = 0.0000
    → Todos são Iris-setosa (puro!)
  Medium: 54 exemplos, entropia = 0.0000
    → Nenhum é Iris-setosa (puro!)
  High: 46 exemplos, entropia = 0.0000
    → Nenhum é Iris-setosa (puro!)


## 4. Construção de Árvore de Decisão

**Estratégia**: Usar o ganho de informação para escolher a melhor feature em cada nó.

### Algoritmo:
1. **Nó Raiz**: Escolher a feature com maior ganho
2. **Particionamento**: Dividir dados pelos valores da feature (Low/Medium/High)
3. **Recursão**: Para cada subconjunto:
   - Se entropia = 0 → nó folha (classe pura)
   - Senão → repetir com features restantes
4. **Critérios de parada**: Entropia = 0 ou não há mais features

### Construção da Árvore de Decisão

**PASSO 1: Nó Raiz**
- Escolher feature com maior ganho: Feature 3 (Petal Length) (0.9183)
- Entropia inicial: 0.9183

**PASSO 2: Particionamento**
- Dividir dados pelos valores de Feature 3 (Petal Length):
  - Low: 50 exemplos → FOLHA: Iris-setosa (todos puros)
  - Medium: 54 exemplos → FOLHA: Outras classes (todos puros)
  - High: 46 exemplos → FOLHA: Outras classes (todos puros)

**PASSO 3: Resultado**
- A árvore seria muito simples neste caso!
- Feature 3 (Petal Length) separa perfeitamente as classes
- Todos os subconjuntos têm entropia = 0 (puros)
- Não é necessário continuar o particionamento

**REGRAS DE DECISÃO EXTRAÍDAS:**
- SE Feature 3 (Petal Length) ≤ 1.90 ENTÃO Iris-setosa
- SE 1.90 < Feature 3 (Petal Length) ≤ 4.35 ENTÃO Outras classes
- SE Feature 3 (Petal Length) > 4.35 ENTÃO Outras classes



## Resumo dos Resultados

### Entropias Calculadas:
- **Conjunto completo**: 0.9183
- **Subconjuntos**: Varia conforme a feature

### Melhor Feature:
- **Feature 3 (Petal Length)** com ganho de **0.9183**
- Oferece separação perfeita das classes

### Árvore de Decisão:
- Estrutura simples com apenas um nível
- Regras claras e interpretáveis
- Classificação perfeita possível
